In [ ]:
from sklearn.metrics import classification_report 
from sklearn.metrics import precision_recall_curve, f1_score
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report 
from sklearn.feature_selection import SelectFromModel  

In [ ]:
def run_pipleline_GridSearchCV(gridserachcv, x_train, y_train, x_test, y_test):

    gridserachcv.fit(x_train, y_train)
    best_params = gridserachcv.best_params_
    print("Best parameters: ", best_params)
    print("Best F1 Score:", gridserachcv.best_score_)
    model1_best = gridserachcv.best_estimator_
    model1_best = model1_best[1:] # remove the pipeline adasyn
    print("run score")
    print(model1_best)

    feature_df = get_feature_importance(model1_best,x_test.columns)

    #SelectFromModel
    x_train_selected = x_train
    x_test_selected = x_test
    if hasattr(model1_best, "feature_importances_") or hasattr(model1_best, "coef_"):
        selector = SelectFromModel(model1_best, threshold='>0', prefit=True)
        x_train_selected = selector.transform(x_train)
        x_test_selected = selector.transform(x_test)

    # x_train2 = x_train.drop(columns= list(feature_df[feature_df['Importance']==0]['Feature']))
    # x_test_selected = x_test.drop(columns= list(feature_df[feature_df['Importance']==0]['Feature']))
    model1_best.fit(x_train_selected, y_train)

    # model1_best.fit(columnsWithoutList(x_train,col_set_feature_engineering), y_train)
    rs_train = run_score_1(model1_best, 'train', x_train_selected, y_train,True)
    rs_test =  run_score_1(model1_best, 'test', x_test_selected, y_test,True)
    rs_train.accuracy_test = rs_test.accuracy
 
    rs_train.accuracy_test = rs_test.accuracy
    rs_train.roc_auc_test = rs_test.roc_auc
    rs_train.coh_kap_test = rs_test.coh_kap
    rs_train.f1_test = rs_test.f1
    rs_train.fper_test = rs_test.fper
    rs_train.tper_test = rs_test.tper
    rs_train.precision_test = rs_test.precision
    rs_train.recall_test = rs_test.recall




    
    return rs_train
    

In [ ]:
class Result_Score:
    def __init__(self):
        self.model = None
        
        self.accuracy = None
        self.roc_auc = None
        self.coh_kap = None
        self.f1 = None
        self.fper = None
        self.tper = None
        self.precision = None
        self.recall = None
        

In [ ]:
class Result_Score:
    def __init__(self):
        self.model = None
        
        self.accuracy = None
        self.roc_auc = None
        self.coh_kap = None
        self.f1 = None
        self.fper = None
        self.tper = None
        self.precision = None
        self.recall = None

        self.accuracy_test = None
        self.roc_auc_test = None
        self.coh_kap_test = None
        self.f1_test = None
        self.fper_test = None
        self.tper_test = None
        self.precision_test = None
        self.recall_test = None
        

In [ ]:

def run_score_1(model, dataType, x, y, verbose=True)->Result_Score:
    
    y_pred = model.predict(x)
    

    accuracy = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    roc_auc = roc_auc_score(y, y_pred) 
    coh_kap = cohen_kappa_score(y, y_pred)
   
    print("---------------------------------------")
    print(dataType)
    print("  Accuracy = {}".format(accuracy))
    print("  F1 Score = {}".format(f1))
    print("  ROC Area under Curve = {}".format(roc_auc))
    print("  Cohen's Kappa = {}".format(coh_kap))
    print(classification_report(y,y_pred,digits=5))
        
    cm = confusion_matrix(y, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix - '+dataType)
    plt.show()

    probs = model.predict_proba(x)  
    print(probs)
    probs = probs[:, 1]  
    fper, tper, thresholds = roc_curve(y, probs) 
    plot_roc_cur(fper, tper)
    
    # Calculate precision, recall, and thresholds
    precision, recall, thresholds = precision_recall_curve(y, probs)
    

    # Plot Precision-Recall curve
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, marker='.', label=model[-1].__class__.__name__)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve\nF1 Score: {:.3f}'.format(f1))
    plt.grid(True)
    plt.legend()
    plt.show()

    rs = Result_Score()
    rs.model = model
    rs.accuracy = accuracy
    rs.roc_auc = roc_auc
    rs.coh_kap = coh_kap
    rs.f1 = f1
    rs.fper = fper
    rs.tper = tper
    rs.precision = precision
    rs.recall = recall

    return rs

In [ ]:
def plot_roc_cur(fper, tper):  
    plt.plot(fper, tper, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [ ]:
def get_feature_importance(modelOrPipeline, x_train_columns):
    
    if hasattr(modelOrPipeline, 'steps'):
        model = modelOrPipeline[-1]
    else:
        model = modelOrPipeline

    importances_df = pd.DataFrame(columns=['Feature','Importance'])
      
    if hasattr(model, "feature_importances_"):
        importances = model.feature_importances_

        # 確保 features 是一維純文字索引
        features = pd.Index(x_train_columns).astype(str)

        # 建立資料表
        importances_df = pd.DataFrame({'Feature': features, 'Importance': importances})
        importances_df = importances_df.sort_values(by='Importance', ascending=False)
        print(importances_df)
        # 畫圖
        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=importances_df.head(20))
        plt.title('Top 20 Feature Importances')
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.show()
    elif  hasattr(model, "coef_"):
        importances = model.coef_[0]
        features = pd.Index(x_train_columns).astype(str)
        importances_df = pd.DataFrame({'Feature': features, 'Importance': importances})
        importances_df = importances_df.reindex(importances_df['Importance'].abs().sort_values(ascending=False).index)
        print(importances_df)
        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=importances_df.head(20))
        plt.title('Top 20 Feature Importances')
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.show()
    return importances_df